In [1]:
import pandas as pd

In [2]:
base_url = "https://en.wikipedia.org/wiki/"
file = "Resources/forbes_billionaires.csv"

In [3]:
#grab billionaries 
bill_df = pd.read_csv(file)

In [4]:
names = bill_df['Name']
names = list(names)
names[0:5]

['Jeff Bezos',
 'Elon Musk',
 'Bernard Arnault & family',
 'Bill Gates',
 'Mark Zuckerberg']

In [5]:
names = [name.split(' &')[0] for name in names]

In [6]:
names = [name.replace(' ', '_') for name in names]
names[0:5]

['Jeff_Bezos', 'Elon_Musk', 'Bernard_Arnault', 'Bill_Gates', 'Mark_Zuckerberg']

In [8]:
list_of_tables = []
for name in names:
    try:
        list_of_tables.append(pd.read_html(base_url + name)[0])
    except Exception as e:
        print(name, '\n', e)

Daniel_Gilbert 
 No tables found
François_Pinault 
 'ascii' codec can't encode character '\xe7' in position 14: ordinal not in range(128)
David_Thomson 
 No tables found
Wang_Wei 
 No tables found
Jim_Simons 
 No tables found
Jiang_Rensheng 
 HTTP Error 404: Not Found
Zhang_Yong 
 No tables found
Richard_Qiangdong_Liu 
 HTTP Error 404: Not Found
Li_Xiting 
 HTTP Error 404: Not Found
Stefan_Persson 
 No tables found
Xu_Hang 
 HTTP Error 404: Not Found
Chen_Bang 
 HTTP Error 404: Not Found
Andrey_Melnichenko 
 No tables found
Ernest_Garcia,_II. 
 HTTP Error 404: Not Found
Thomas_Frist,_Jr. 
 HTTP Error 404: Not Found
Zuo_Hui 
 HTTP Error 404: Not Found
Cheng_Yixiao 
 HTTP Error 404: Not Found
Wang_Liping 
 No tables found
Huang_Shilin 
 HTTP Error 404: Not Found
Dang_Yanbao 
 HTTP Error 404: Not Found
Hank 
 No tables found
Lu_Zhongfang 
 HTTP Error 404: Not Found
Yu_Renrong 
 HTTP Error 404: Not Found
Liu_Yonghao 
 No tables found
Bobby_Murphy 
 No tables found
Cen_Junda 
 HTTP Error 40

Vinod 
 No tables found
Koon_Poh_Keong 
 HTTP Error 404: Not Found
Paul_Xiaoming_Lee 
 HTTP Error 404: Not Found
Li_Xiang 
 No tables found
Melissa_Ma 
 HTTP Error 404: Not Found
Georg_Nemetschek 
 HTTP Error 404: Not Found
Masahiro_Noda 
 HTTP Error 404: Not Found
Trevor_Rees-Jones 
 No tables found
Wu_Jianshu 
 HTTP Error 404: Not Found
Ye_Xiaoping 
 HTTP Error 404: Not Found
Che_Jianxing 
 HTTP Error 404: Not Found
Beatriz_Davila_de_Santo_Domingo 
 HTTP Error 404: Not Found
Rafael_Del_Pino 
 No tables found
Du_Weimin 
 HTTP Error 404: Not Found
Jin_Baofang 
 HTTP Error 404: Not Found
Stephen_Mandel,_Jr. 
 HTTP Error 404: Not Found
Maximilian_Viessmann 
 HTTP Error 404: Not Found
Wang_Linpeng 
 HTTP Error 404: Not Found
Wu_Zhigang 
 HTTP Error 404: Not Found
Yi_Zheng 
 HTTP Error 404: Not Found
Feng_Hailiang 
 HTTP Error 404: Not Found
Gao_Dekang 
 HTTP Error 404: Not Found
Jeff_Green 
 No tables found
Lai_Jianping 
 HTTP Error 404: Not Found
Hans_Georg_Naeder 
 HTTP Error 404: Not F

Caroline_Hagen_Kjos 
 HTTP Error 404: Not Found
Anthony_Hsieh 
 HTTP Error 404: Not Found
Raj_Kumar 
 No tables found
Adam_Kwok 
 HTTP Error 404: Not Found
Lai_Shixian 
 HTTP Error 404: Not Found
K.C._Liu 
 HTTP Error 404: Not Found
Yuji_Otsuka 
 HTTP Error 404: Not Found
Ren_Jianhua 
 HTTP Error 404: Not Found
Alexey_Repik 
 HTTP Error 404: Not Found
Shum_Chiu_Hung 
 HTTP Error 404: Not Found
Stephen_Smith 
 No tables found
Dan_Snyder 
 No tables found
Evan_Williams 
 No tables found
Wu_Guangming 
 HTTP Error 404: Not Found
Yang_Yuanxi 
 HTTP Error 404: Not Found
Yu_Qibing 
 HTTP Error 404: Not Found
Zhaojiang_Zhu 
 HTTP Error 404: Not Found
William_Berkley 
 No tables found
Andrei_Bokarev 
 HTTP Error 404: Not Found
Byju_Raveendran_and_Divya_Gokulnath 
 HTTP Error 404: Not Found
Chang_Jing 
 HTTP Error 404: Not Found
Jose_Luis_Cutrale 
 HTTP Error 404: Not Found
Hans-Werner_Hector 
 HTTP Error 404: Not Found
Hans_Langer 
 HTTP Error 404: Not Found
Lee_Hae-jin 
 HTTP Error 404: Not Fo

KeyboardInterrupt: 

In [9]:
len(list_of_tables)

960

In [17]:
list_of_tables[5]

,Warren Buffett,Warren Buffett.1
0,Buffett in 2010,Buffett in 2010
1,Born,"Warren Edward Buffett (age 90)Omaha, Nebraska,..."
2,Education,University of PennsylvaniaUniversity of Nebras...
3,Occupation,Businessmaninvestorphilanthropist
4,Years active,1951–present
5,Known for,Leadership of Berkshire Hathaway with Charlie ...
6,Net worth,US$110.4 billion (As of 5 June 2021[1]
7,Spouse(s),Susan Thompson ​ ​(m. 1952; died 2004)​ Astri...
8,Children,Susan Alice BuffettHoward Graham BuffettPeter ...
9,Parent(s),Howard BuffettLeila Stahl Buffett
